<a href="https://colab.research.google.com/github/Alejandro-Huaman/cc41_tf_201910144_201910708_20161c743_202010807_201710251/blob/main/Algoritmo%20Dijkstra/DijkstraAlmacenesPuntosCercanos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import urllib.request
import math
import heapq as hq

### Leer y Guardar
- Puntos de Almacen
- Puntos de Entrega
- Lista de Adyacencia

In [6]:
def readAdjl(fn):
  with urllib.request.urlopen(fn) as f:
    L = []
    for line in f:
        L.append(list(map(int, line.strip().split())))
  return L

In [7]:
def coordToIndex(coord, size):
    return coord[0] * size + coord[1]

In [8]:
def readCoords(fn, size):
  arr = []

  with urllib.request.urlopen(fn) as f:
    for line in f:
      coord = list(map(int, (line.decode("utf-8").strip().split(";"))))
      arr.append(coordToIndex(coord, size))
  
  return arr

In [9]:
urlAdjList = 'https://raw.githubusercontent.com/Alejandro-Huaman/cc41_tf_201910144_201910708_20161c743_202010807_201710251/main/Lista%20de%20Adyacencia/adjList.in'
urlAlmacen = 'https://raw.githubusercontent.com/Alejandro-Huaman/cc41_tf_201910144_201910708_20161c743_202010807_201710251/main/almacenes.csv'
urlEntrega = 'https://raw.githubusercontent.com/Alejandro-Huaman/cc41_tf_201910144_201910708_20161c743_202010807_201710251/main/puntos_entrega.csv'

G = readAdjl(urlAdjList)
P_Almacen = readCoords(urlAlmacen, 1000)
P_Entrega = readCoords(urlEntrega, 1000)

### Puntos de entrega más cercanos por cada almacén

In [10]:
#Convert index to coordinates
def indexToCoord(index, size):
    x = index // size
    y = index % size
    return (x, y)

In [11]:
#Get distance between two points
def getDistance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

In [12]:
def closest_p_a_to_p_e(p_e):
  x_e, y_e = indexToCoord(p_e, 1000)
  closest_p_a = P_Almacen[0]
  for p_a in P_Almacen:
    x_a, y_a = indexToCoord(p_a, 1000)
    x_closest, y_closest = indexToCoord(closest_p_a, 1000)
    if getDistance(x_a,y_a,x_e,y_e) < getDistance(x_closest,y_closest,x_e,y_e):
      closest_p_a = p_a
  return closest_p_a

In [ ]:
#Puntos de entrega más cercanos por cada almacén

dict_p_almacen = {}

for p_e in P_Entrega:
    closest_p_a = closest_p_a_to_p_e(p_e)
    if not closest_p_a in dict_p_almacen:
        dict_p_almacen[closest_p_a] = [p_e]
    else:
        dict_p_almacen[closest_p_a].append(p_e)

total_p_e_counter = 0
for count, p_a in enumerate(P_Almacen):
    if not p_a in dict_p_almacen:
        dict_p_almacen[p_a] = []
    print(f"Almacen {count} ({p_a}): {dict_p_almacen[p_a]})")
    print(f"Contiene: {len(dict_p_almacen[p_a])} puntos de entrega")
    total_p_e_counter += len(dict_p_almacen[p_a])

print(f"\nEl total de puntos de entrega es: {total_p_e_counter}")

### Dijkstra - **Old Version**
Este es el algoritmo realizado para los antiguos hitos. Por favor, pasar a la siguiente sección para ejecutar la versión actual.

In [ ]:
def dijkstra(G, s):
  n = len(G)
  visited = [False]*n
  path = [None]*n
  cost = [math.inf]*n
  cost[s] = 0
  queue = [(s, 0)]
  while queue:
    u, g_u = hq.heappop(queue)
    if not visited[u]:
      visited[u] = True        
      for v in G[u]:
        f = g_u + 1
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (v, f))
  return path

In [ ]:
#Devuelve el camino (arreglo) hasta el primer punto
def WayToPoint(p_e, init, indexEntrega, indexAlmacen):
  tr = [p_e] #trayectoria
  aux = path[p_e] - p_e

  while (p_e != None):
    parent = path[p_e]
    if parent != None and parent - p_e != aux:
      tr.append(p_e)
      break
    p_e = parent
  
  tr.append(init)
  tr.reverse()
  #Get total distance
  distance = 0
  for i in range(len(tr) - 1):
    x1, y1 = indexToCoord(tr[i], 1000)
    x2, y2 = indexToCoord(tr[i+1], 1000)
    distance += getDistance(x1, y1, x2, y2)

  #change current element minorDistance
  if distance < minorDistance[indexEntrega][1]:
    targetAlmacen = minorDistance[indexEntrega][0]
    if targetAlmacen != -1:
      listArrCaminos[targetAlmacen][indexEntrega] = []
    minorDistance[indexEntrega] = (indexAlmacen, distance)
    return tr
  return []


In [ ]:
#Crear arreglo de distancias hacia los almacenes
minorDistance = [[-1,float('inf')]]*len(P_Entrega)
listArrCaminos = []

for index in range(len(P_Almacen)):
  p_a = P_Almacen[index]
  path = dijkstra(G, p_a)
  arrCaminos = []
  for count, p_e in enumerate(P_Entrega):
   arr = WayToPoint(p_e, p_a, count, index)
   arrCaminos.append(arr)
  listArrCaminos.append(arrCaminos)

fixedListArrCaminos = []
for index, arr in enumerate(listArrCaminos):
  #delete empty arrays in arr
  fixedArr = []
  for path in arr:
    if path != None and len(path) > 0:
      fixedArr.append(path)
  fixedListArrCaminos.append(fixedArr)

#Print fixedListArrCaminos
total = 0
for index, arr in enumerate(fixedListArrCaminos):
  total += len(arr)
  print(f"Almacen {index} ({P_Almacen[index]}) : {arr}")

print("La cantidad de puntos de entrega es: " + str(total))

### Dijkstra Grupal - **New Version**
En este Dijkestra empleamos los diccionarios, lo que nos facilitó considerablemente el utilizar los algoritmos por secciones.

In [14]:
def dijkstraDict(G, s):
  visited = {}
  path = {}
  cost = {}
  for key in G.keys():
    visited[key] = False
    path[key] = None
    cost[key] = math.inf

  cost[s] = 0
  queue = [(s, 0)]
  while queue:
    u, g_u = hq.heappop(queue)
    if not visited[u]:
      visited[u] = True        
      for v in G[u]:
        f = g_u + 1
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (v, f))
  return path

In [15]:
def create_adjacency_list(p_e_list, init):
    adjacency_list = {}
    max_x, max_y = indexToCoord(init,1000)
    min_x, min_y = indexToCoord(init,1000)
    for p_e in p_e_list:
        e_x, e_y = indexToCoord(p_e, 1000)
        if e_x > max_x:
            max_x = e_x
        elif e_x < min_x:
            min_x = e_x
        if e_y > max_y:
            max_y = e_y
        elif e_y < min_y:
            min_y = e_y
    

    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            adjacency_list[coordToIndex((x,y),1000)] = []

    for point in adjacency_list:
        x, y = indexToCoord(point, 1000)
        left_point = coordToIndex((x - 1, y), 1000)
        right_point = coordToIndex((x + 1, y), 1000)
        up_point = coordToIndex((x, y + 1), 1000)
        down_point = coordToIndex((x, y - 1), 1000)

        if left_point in adjacency_list:
            adjacency_list[point].append(left_point)
        if right_point in adjacency_list:
            adjacency_list[point].append(right_point)
        if up_point in adjacency_list:
            adjacency_list[point].append(up_point)
        if down_point in adjacency_list:
            adjacency_list[point].append(down_point)

    return adjacency_list
      


In [16]:
#Devuelve el camino (arreglo) hasta el primer punto
def WayToPoint(path, p_e, init):
  #print(p_e)
  tr = [] #trayectoria
  aux = path[p_e] - p_e

  change_x = 0
  change_y = 0

  while p_e != None:
    if path[p_e] == -1: break
    
    parent = path[p_e]
    if parent != None:
      e_x, e_y = indexToCoord(p_e, 1000)
      p_x, p_y = indexToCoord(parent, 1000)
      aux_change_x = change_x
      aux_change_y = change_y
      if e_x != p_x:
        change_x = 1
        if change_x != aux_change_x:
          change_y = 0
          tr.append(p_e)
      if e_y != p_y:
        change_y = 1
        if change_y != aux_change_y:
          change_x = 0
          tr.append(p_e)
    p_e = parent
  
  tr.append(init)
  tr.reverse()

  return tr


In [ ]:
for index, p_a in enumerate(P_Almacen):
    adjList = create_adjacency_list(dict_p_almacen[p_a], p_a)
    path = dijkstraDict(adjList, p_a)
    arrCaminos = []
    for p_e in dict_p_almacen[p_a]:
         arrCaminos.append(WayToPoint(path, p_e, p_a))
    print(f"Almacen {index} ({p_a}) : {arrCaminos}")


### Algoritmos de integrantes
En esta sección se encuentran los algoritmo por integrantes.

In [36]:
#INTEGRANTE 0 - Galindo Alvarez, Franco - u202010807
def bellmanFordDict(G, p_a):
  n = len(G)
  cost = {}
  path = {}

  for key in G.keys():
    path[key] = None
    cost[key] = math.inf

  cost[p_a] = 0

  for _ in range(n-1):
    for u in G.keys():
      for v in G[u]:
        if cost[u] + 1 < cost[v]:
          cost[v] = cost[u] + 1
          path[v] = u

  # check negative cycle
  for u in G.keys():
    for v in G[u]:
      if cost[u] + 1 < cost[v]:
        return None # negative cycle exists
        
  return path


In [18]:
#INTEGRANTE 1 - Huamán Ruiz, Alejandro Sebastián - u201910144
def primDict(G,p_a):
  visited = {}
  path = {}
  cost = {}
  for key in G.keys():
    visited[key] = False
    path[key] = None
    cost[key] = math.inf
  cost[p_a] = 0
  q = [(0, p_a)]
  while q:
    _, u = hq.heappop(q)
    if not visited[u]:
      visited[u] = True
      for v in G[u]:
        if not visited[v] and 1 < cost[v]:
          cost[v] = 1
          path[v] = u
          hq.heappush(q, (1, v))

  return path

In [19]:
#INTEGRANTE 2 - Rebaza Maticorena, Giusen Eduardo - U201910708
class DisjointSet:
  S = {}
  # todos los métodos tiene el parametro oculto self
  # self === this
  # Constructor!
  def __init__(self,G):
    for key in G.keys(): 
      self.S[key] = -1

  ## Acuerdo de personas honorables: __algo: privado, _algo: protegido, algo: publico.
  def find(self, x):
    if self.S[x] < 0:
      return x
    p = self.find(self.S[x])
    self.S[x] = p
    return p

  def union(self, x, y):
    Xr = self.find(x)
    Yr = self.find(y)
    if Xr != Yr:
      if self.S[Xr] < self.S[Yr]:
        self.S[Xr] += self.S[Yr]
        self.S[Yr] = Xr
      else:
        self.S[Yr] += self.S[Xr]
        self.S[Xr] = Yr

  def isSameSet(self, x, y):
    return self.find(x) == self.find(y)
  
def kruskal(G):
  n = len(G)
  edges = []
  for u in G:
    for v in G[u]:
      hq.heappush(edges, (1, u, v))

  uf = DisjointSet(G)

  T = []
  path = {}
  for key in G.keys():
    path[key] = -1
    
  while edges and n > 0:
    _, u, v = hq.heappop(edges)
    if not uf.isSameSet(u, v):
      uf.union(u, v)
      T.append((u, v, 1))
      if path[u] < 0:
        path[u] = v
      else:
        path[v] = u
      n -= 1

  return path

In [20]:
#INTEGRANTE 3 - Valdivia Zurita, Ernesto Miguel - u201710251
def dfsDict(G, p_a, limit):
  visited = {}
  path = {}
  reached = {}
  for key in G.keys():
    visited[key] = False
    path[key] = None
    reached[key] = False

  def _dfs(u, limit):
    if limit > 0:
        if not visited[u]:
            visited[u] = True
            for v in G[u]:
              if not visited[v]:
                path[v] = u
                reached[v] = True
                _dfs(v, limit-1)
                
  reached[p_a]= True              
  _dfs(p_a, limit)
     
  return path

In [21]:
#INTEGRANTE 4 - Peralta Saravia, Jose Eduardo - U20161C743

def bfs(G, s):                #(grafo, vertice de origen s)
  visitado = {}
  path = {}
  for key in G.keys():
    visitado[key] = False
    path[key] = None

  queue = [s]
  visitado[s] = True

  while queue:                #mientras cola sea mayor a 0
    u = queue.pop(0)                #popleft() -> elimina 1er elemento de la lista       
    for v in G[u]:            #v es el vecino de u
      if not visitado[v]:
        visitado [v] = True
        path[v] = u           #padre de v sera u
        queue.append(v)
  return path

### Regiones por integrante

In [22]:
def getInicioFin(integrante):
    cantAlmPorIntegrante = len(P_Almacen)/5
    inicio = cantAlmPorIntegrante*integrante
    fin = inicio + cantAlmPorIntegrante
    return inicio, fin

In [ ]:
#INTEGRANTE 0 - Galindo Alvarez, Franco - u202010807
inicio, fin = getInicioFin(0)

for index, p_a in enumerate(P_Almacen):
    if index <= inicio or index > fin: continue
    adjList = create_adjacency_list(dict_p_almacen[p_a], p_a)
    path = bellmanFordDict(adjList, p_a) #REEMPLAZAR POR TU ALGORITMO
    arrCaminos = []
    for p_e in dict_p_almacen[p_a]:
         arrCaminos.append(WayToPoint(path, p_e, p_a))
    print(f"Almacen {index} ({p_a}) : {arrCaminos}")
    break

In [ ]:
#INTEGRANTE 1 - Huamán Ruiz, Alejandro Sebastián - u201910144 
inicio, fin = getInicioFin(1)

for index, p_a in enumerate(P_Almacen):
    if index <= inicio or index > fin: continue
    adjList = create_adjacency_list(dict_p_almacen[p_a], p_a)
    path = primDict(adjList, p_a)
    arrCaminos = []
    for p_e in dict_p_almacen[p_a]:
         arrCaminos.append(WayToPoint(path, p_e, p_a))
    print(f"Almacen {index} ({p_a}) : {arrCaminos}")

In [ ]:
#INTEGRANTE 2 - Rebaza Maticorena, Giusen Eduardo - U201910708
inicio, fin = getInicioFin(2)

for index, p_a in enumerate(P_Almacen):
    if index <= inicio or index > fin: continue
    adjList = create_adjacency_list(dict_p_almacen[p_a], p_a)
    path = kruskal(adjList) 
    arrCaminos = []
    for p_e in dict_p_almacen[p_a]:
        arrCaminos.append(WayToPoint(path, p_e, p_a))
    print(f"Almacen {index} ({p_a}) : {arrCaminos}")
    

In [ ]:
#INTEGRANTE 3 - Valdivia Zurita, Ernesto Miguel - u201710251
inicio, fin = getInicioFin(3)

for index, p_a in enumerate(P_Almacen):
    if index <= inicio or index > fin: continue
    adjList = create_adjacency_list(dict_p_almacen[p_a], p_a)
    path = dfsDict(adjList, p_a, 1500) #REEMPLAZAR POR TU ALGORITMO
    arrCaminos = []
    for p_e in dict_p_almacen[p_a]:
         arrCaminos.append(WayToPoint(path, p_e, p_a))
    print(f"Almacen {index} ({p_a}) : {arrCaminos}")

In [ ]:
#INTEGRANTE 4 - Peralta Saravia, Jose Eduardo - U20161C743
inicio, fin = getInicioFin(4)

for index, p_a in enumerate(P_Almacen):
    if index <= inicio or index > fin: continue
    adjList = create_adjacency_list(dict_p_almacen[p_a], p_a)
    path = bfs(adjList, p_a) #REEMPLAZAR POR TU ALGORITMO
    arrCaminos = []
    for p_e in dict_p_almacen[p_a]:
      arrCaminos.append(WayToPoint(path, p_e, p_a))
    print(f"Almacen {index} ({p_a}) : {arrCaminos}")